# Статистика

## Описательная статистика

Размах, среднее, мода, медиана

In [1]:
from typing import List
from enum import Enum

def amount(numbers: List[float]) -> int:
    ''' Количество элементов '''
    return len(numbers)

def mean(numbers: List[float]) -> float:
    ''' Среднее арифметическое '''
    sum: float = 0
    for i in numbers:
        sum += i
    return sum / len(numbers)

def min_value(numbers: List[float]) -> float:
    ''' Минимальное значение '''
    return min(numbers)

def max_value(numbers: List[float]) -> float:
    ''' Максимальное значение '''
    return max(numbers)

Дисперсия и стандартное отклонение

In [2]:
def variance(numbers: List[float]) -> float:
    ''' Дисперсия '''
    return sum([(i-mean(numbers))**2 for i in numbers]) * 1/(len(numbers)-1)

def std(numbers: List[float]) -> float:
    """ Стандартное отклонение """
    return variance(numbers) ** 0.5

In [3]:
# Пример

dataset = [1, 5, 2, 7, 1, 9, 3, 8, 5, 9, 10, 11, 12, 13, 14, 15, 21, 17, 17, 16]
print("Количество элементов:", amount(dataset))
print("Среднее:", mean(dataset))
print("Минимум:", min_value(dataset))
print("Максимум:", max_value(dataset))
print("Дисперсия:", variance(dataset))
print("Стандартное отклонение:", std(dataset))

Количество элементов: 20
Среднее: 9.8
Минимум: 1
Максимум: 21
Дисперсия: 34.16842105263158
Стандартное отклонение: 5.845376040310117


## Z-значение и Z-преобразование

In [4]:
def z_value(value: float, mean: float, std: float) -> float:
    ''' Расчет z-значения '''
    return (value - mean) / std

def z_values(numbers: List[float]) -> List[float]:
    ''' !TODO '''
    z_values: List[float] = []

    for item in numbers:
        z_val = z_value(item)
        z_values.append(z_val)

    return z_values


def se(std: float, elements_amount: int) -> float:
    '''
    Стандартная ошибка среднего

    Args:
        std (float): Выборочное стандартное отклонение
        elements_amount (int): Количество элементов в выборке
    
    Returns:
        (float): Значение стандартной ошибки среднего
    '''
    return std / elements_amount ** 0.5

## Проверка гипотез

### Доверительный интервал и p-уровень значимости

**Доверительный интервал**. Мы не можем по выборке знать выбранную характеристику всей генеральной совокупности, но можем рассчитать такой интервал, в который с высокой вероятностью попадает интересующий нас параметр.

In [5]:
class ConfidenceLevel(Enum):
    '''
    Значения доверительных уровней
    '''
    
    LVL_90 = 1.64  # Доверительный уровень 90% (1,64 стандартных ошибок)
    LVL_95 = 1.96  # Доверительный уровень 95% (1,96 стандартных ошибок)
    LVL_99 = 2.58  # Доверительный уровень 99% (2,58 стандартных ошибок)

def confidence_interval(
    mean: float,
    se: float,
    confidence_level: ConfidenceLevel
) -> float:
    '''
    Доверительный интервал

    Args:
        mean (float): Выборочное среднее
        se (float): Стандартная ошибка среднего
        confidence_level (ConfidenceLevel): Доверительный уровень

    Returns:
        List[float]: Левая и правая границы доверительного интервала
    '''
    
    ci_border_min: float = mean - se * confidence_level.value
    ci_border_max: float = mean + se * confidence_level.value
    return [ci_border_min, ci_border_max]



**p-уровень значимости.**

Простыми словами: это вероятность того, что полученные нами в ходе эксперимента результаты - случайны.

p > 0.05 -> недостаточно оснований, чтобы отклонить нулевую гипотезу

p < 0.05 -> можно принимать альтернативную гипотезу

**Задача**

_Есть лекарство, принимая которое, пациенты выздоравливают в среднем за 20 дней.
Мы тестировали новое лекарство на выборке в 64 человека и заметили, что пациенты, принимая его, выздоравливают в среднем за 18.5 дней при стандартном отклонении равным 4._

_Какой вывод можно сделать на основе этих данных? Достаточно ли оснований, чтобы принять гипотезу о том, что наше лекарство, действительно уменьшает период лечения?_

In [6]:
'''
H0: количество дней до выздоровления = 20 и не зависит от нового препарата
H1: количество дней до выздоровления != 20 и новый препарат, действительно, повлиял
'''

mean_old: float = 20  # выборочное среднее до эксперимента (среднее ген. совокупности)

n: int = 64  # количество экспериментов
mean_new: float = 18.5  # выборочное среднее после проведения эксперимента
sd: float = 4  # стандартное отклонение результатов в выборке


'''
Предположим, что H0 - верна. 
Тогда, согласно ЦПТ, выборочные средние будут распределены нормально вокруг среднего ГС.
При этом, будет стандартная ошибка
'''

std_err = se(sd, n)
print("Стандартная ошибка среднего:", std_err)

'''
Определим, насколько далеко наше выборочное среднее отклонилось от предполагаемого
среднего ГС в единицах стандартного отклонения
Для этого, сделаем Z-преобразование
'''

z_val = z_value(mean_new, mean_old, std_err)
print("Результат z-преобразования:", z_val)


'''
Это значит, что если бы в ГС среднее значение на самом деле равнялось бы 20,
то наше выборочное среднее отклонилось бы от среднего ГС на -3 стандартных отклонения
в левую сторону.

Дальше нужно рассчитать вероятность такого или еще более сильно выраженного отклонения
от среднего значения.
Расчет можно произвести по ссылке:  https://gallery.shinyapps.io/dist_calc/

Эта вероятность p равляется примерно 0.003
'''

pass


Стандартная ошибка среднего: 0.5
Результат z-преобразования: -3.0


**Задача**

_В среднем слушатели курса по введению в статистику набирают 115 баллов._

_Однако, в 2015 году средний балл  случайно выбранных 144 участников составил 118 со стандартным отклонением равным 9._

_Рассчитайте p уровень значимости для проверки нулевой гипотезы о том, что среднее значение баллов в 2015 году равняется 115._

In [7]:
'''
H0: количество баллов = 115 и не зависит от года
H1: количество баллов != 115 и зависит от года
'''

mean_old: float = 115  # выборочное среднее до эксперимента (среднее ген. совокупности)

n: int = 144  # количество экспериментов
mean_new: float = 118  # выборочное среднее после проведения эксперимента
sd: float = 9  # стандартное отклонение результатов в выборке

# Расчитаем стандартную ошибку среднего
std_err = se(sd, n)
print("Стандартная ошибка среднего:", std_err)

# Сделаем z-преобразование
z_val = z_value(mean_new, mean_old, std_err)
print("Результат z-преобразования:", z_val)

'''
Идем в калькулятор по ссылке: https://gallery.shinyapps.io/dist_calc/
Расчитываем вероятность p = 6.33e-05
'''

pass


Стандартная ошибка среднего: 0.75
Результат z-преобразования: 4.0


## T-распределение

Ситуация: у нас маленькое количество наблюдений (<30). В этом случае, ЦПТ не работает и выборочные средние не распределяются нормально.
В этом случае используется распределение Стьюдента (t-распределение)

### Сравнение средних и t-критерий

Критерий, который позволяет сравнить между собой два выборочных средних.

In [8]:
def sed(std_1: float, std_2: float, n_1: int, n_2: int) -> float:
    '''
    Стандартная ошибка разницы

    Args:
        std_1 (float): Стандартное отклонение первой выборки
        std_2 (float): Стандартное отклонение второй выборки
        n_1 (int): Количество элементов первой выборки
        n_2 (int): Количество элементов второй выборки

    Returns:
        (float): Стандартная ошибка разницы
    '''
    return (std_1 ** 2 / n_1 + std_2 ** 2 / n_2) ** 0.5


def t_criterion(mean_1: float, mean_2: float, sed:float) -> float:
    '''
    t-критерий (t-статистика)

    Args:
        mean_1 (float): Среднее первой выборки
        mean_2 (float): Среднее второй выборки
        sed (float): Стандартная ошибка разницы

    Returns:
        (float): t-критерий
    '''

    return (mean_1 - mean_2) / sed


def t_confidence_interval(
    mean: float,
    se: float,
    confidence_level: ConfidenceLevel
) -> float:
    '''
    Доверительный интервал

    Args:
        mean (float): Выборочное среднее
        se (float): Стандартная ошибка среднего
        confidence_level (ConfidenceLevel): Доверительный уровень

    Returns:
        List[float]: Левая и правая границы доверительного интервала
    '''
    
    ci_border_min: float = mean - se * confidence_level.value
    ci_border_max: float = mean + se * confidence_level.value
    return [ci_border_min, ci_border_max]



**Задача**

_Рассчитайте доверительный интервал основываясь на знании t - распределения для среднего значения температуры плавления ДНК у какого-то вида:_

_x (среднее) = 89.9, sd = 11.3, n = 20_

_Для этого определите, в каком диапазоне находится 95 % наблюдений у соответствующего t - распределения (df = n -1) и используйте это значение вместо коэффициента 1,96._

In [9]:
mean_val: float = 89.9  # Выборочное среднее
sd: float = 11.3  # Стандартное отклонение
n: int = 20  # Количество наблюдений

# Определяем количество степеней свободы
df: int = n - 1

# Для этого значения степеней свободы 
# по специальной таблице определяем значение коэффициента, который будем
# использовать для расчета доверительного интервала при t-распределении
coeff: float = 2.093

# Расчитаем стандартную ошибку среднего
std_err = se(sd, n)

# Расчитываем границы доверительного интервала
ci_border_min: float = mean_val - std_err * coeff
ci_border_max: float = mean_val + std_err * coeff

print("Доверительный интервал:", [ci_border_min, ci_border_max])


Доверительный интервал: [84.61149798709502, 95.18850201290499]


**Задача**

_Первые премии Оскар за лучшую мужскую и женскую роль были вручены в 1929. Данные гистограммы демонстрируют распределение возраста победителей с 1929 по 2014 год (100 мужчин, 100 женщин). Используя t - критерий проверьте, можно ли считать наблюдаемые различия в возрасте между лучшими актрисами и актерами  статистически достоверными._

- _Средний возраст мужчин равен 45, sd = 9_
- _Средний возраст женщин равен 34, sd = 10._

In [10]:
mean_m: float = 45  # Средний возраст мужчин
sd_m: float = 9  # Стандартное отклонение по выборке из мужчин
n_m: int = 100  # Количество наблюдений

mean_f: float = 34  # Средний возраст женщин
sd_f: float = 10  # Стандартное отклонение по выбооке из женщин
n_f: int = 100  # Количество наблюдений

# Расчитаем стандатную ошибку разницы
std_err_diff = sed(sd_m, sd_f, n_m, n_f)
print("Стандартная ошибка разницы:", std_err_diff)

# Расчитаем t-критерий
t_crt = t_criterion(mean_m, mean_f, std_err_diff)
print("t-критерий:", t_crt)

# Расчитаем число степеней свободы
df: int = n_m + n_f - 2

# Для этого количества степеней свободы найдет табличное значение коэффициента
t_coeff = 1.972

# Специальным калькулятором считаем вероятность выхода за пределы интервала
p: float = 3.48e-14

# Либо же, можем решить задачу по следующей логике:
# t наше = 8,2 (11/корень(181/100))
# t табл. = 1,972 (при p = 0,05 и df = 200, так как 198 в таблице нет, различие будет невелико)
# t наше > t табл. -> нулевую гипотезу отклоняем


Стандартная ошибка разницы: 1.3453624047073711
t-критерий: 8.176235608718828


## Однофакторный дисперсионный анализ

In [11]:
def sum_sqrs(numbers: List[float]) -> float:
    '''
    Сумма квадратов выборки

    Args:
        numbers (List[float]): Данные
    
    Returns:
        (float): Общая сумма квадратов
    '''
    sum_sq: float = 0.0

    for i in numbers:
        sum_sq += (i - mean(numbers)) ** 2

    return sum_sq


def sst(*numbers: List[float]) -> float:
    '''
    Обшая сумма квадратов

    Args:
        *numbers (List[float]): один или несколько массивов

    Returns:
        sst (float): Общая сумма квадратов
    '''

    sst: float = 0

    # Сформируем из групп один список
    data_all: List[float] = []
    for group in numbers:
        for item in group:
            data_all.append(item)

    # Находим SST
    sst = sum_sqrs(data_all)

    return sst


def ssw(*numbers: List[float]) -> float:
    '''
    Внутригрупповая сумма квадратов (Square Sum Within)
    Берем несколько массивов с числами, для каждого массива вычисляем
    общую сумму квадратов его элементов и складываем эти суммы.

    Args:
        *numbers (List[float]): один или несколько массивов

    Returns:
        ssw (float): Внутригрупповая сумма квадратов
    '''
    
    ssw: float = 0

    for data_group in numbers:
        ssw += sum_sqrs(data_group)
    
    return ssw


def ssb(*numbers: List[float]) -> float:
    '''
    Межгрупповая сумма квадратов (Square Sum Between)
    '''

    ssb: float = 0
    
    groups_amount: int = len(numbers)
    group_elements_amount: int = 0
    group_elements_sum: float = 0
    all_mean: float = 0
    for data_group in numbers:
        group_elements_amount += len(data_group)
        for item in data_group:
            group_elements_sum += item
        all_mean = group_elements_sum / group_elements_amount
        

    for data_group in numbers:
        data_group_mean = mean(data_group)
        ssb += groups_amount * ((data_group_mean - all_mean) ** 2)
    
    return ssb


In [12]:
'''
Пример

Есть три выборки
Нулевая гипотеза: Средние всех трех выборок равны
Альтернативная гипотеза: Средние всех трех выборок не равны
'''

sel1 = [3,1,2]
sel2 = [5,3,4]
sel3 = [7,6,5]
print("Общая сумма квадратов:", sst(sel1, sel2, sel3))
print("Внутригрупповая сумма квадратов:", ssw(sel1, sel2, sel3))
print("Межгрупповая сумма квадратов:", ssb(sel1, sel2, sel3))

'''
Число степеней свободы для SSB = число групп - 1
Число степеней свободы для SSW = общее количество элементов - количество групп
'''

# Расчитываем F-значение
f_value = (ssb(sel1, sel2, sel3) / 2) / (ssw(sel1, sel2, sel3) / 6)
print("F-значение:", f_value)

# Далее, расчитываем в специальном калькуляторе p-уровень значимости при помощи F-критерия

Общая сумма квадратов: 30.0
Внутригрупповая сумма квадратов: 6.0
Межгрупповая сумма квадратов: 24.0
F-значение: 12.0


### Множественные сравнения в дисперсном анализе

In [13]:
def mse(*numbers: List[float]) -> float:
    '''
    Средний квадрат отклонений (Mean Square Error)

    Args:
        *numbers (List[float]): Один или нескалько массивов

    Returns:
        mse (float): Средний квадрат отклонений для всех групп
    '''

    groups_amount: int = len(numbers)
    items_amount = sum(len(group) for group in numbers)

    mse = ssw(*numbers) / (items_amount - groups_amount)

    return mse


def bonferroni(p_level: float, comparisons: int) -> float:
    '''
    Поправка Бонферрони

    Args:
        p_level (float): p-уровень значимости, с которым мы работаем
        comparisons (int): Количество сравнений

    Returns:
        (float): Скорректированный показатель
    '''

    return p_level / comparisons


def q_criterion(numbers_1: List[float], numbers_2: List[float]) -> float:
    '''
    Критерий Тьюки (q-критерий) для пары выборок

    Args:
        numbers_1 (List[float]): Первая выборка
        numbers_2 (List[float]): Вторая выборка

    Returns:
        q_criterion (float): q-критерий для пары выборок
    '''

    q_criterion: float = (mean(numbers_1) - mean(numbers_2)) / mse(numbers_1, numbers_2)

    return q_criterion



In [14]:
# Пример
sel1 = [3, 1, 2, 2, 5, 3, 11, 4, 7, 8, 9]
sel2 = [11, 11, 12, 12, 15, 13, 11, 14, 17, 18, 14]
#sel2 = [5, 3, 4, 4, 6, 7, 8, 8, 8, 12, 1, 1]

print("Средний квадрат отклонений:", mse(sel1, sel2))
print("q-критерий:", q_criterion(sel1, sel2))


Средний квадрат отклонений: 8.336363636363636
q-критерий: -1.014176663031625


## Корреляция

In [15]:
def covariation(n1: List[float], n2: List[float]) -> float:
    ''' 
    Ковариация 
    
    Args:
        n1 (List[float]): Первая последовательность
        n2 (List[float]): Вторая последовательность

    Returns:
        covariation (float): Значение ковариации
    '''
    
    n1_mean: float = mean(n1)  # Среднее в первой последовательности
    n2_mean: float = mean(n2)  # Среднее во второй последовательности

    summ_n: float = 0  # Сумма произведений отклонений от средних значений

    for i in range(0, len(n1)):
        summ_n += (n1[i] - n1_mean) * (n2[i] - n2_mean)

    covariation: float = summ_n / (len(n1) - 1)  # Среднее значение произведения отклонений от средних значений

    return covariation


def correlation_pearson(n1: List[float], n2: List[float]) -> float:
    '''
    Коэффициент корреляции Пирсона

    Args:
        n1 (List[float]): Первая последовательность
        n2 (List[float]): Вторая последовательность

    Returns:
        correlation (float): Коэффициент корреляции
    '''

    correlation: float = covariation(n1, n2) / (std(n1) * std(n2))

    return correlation

In [16]:
# Пример
x = [4, 5, 2, 3, 1]
y = [2, 1, 4, 3, 5]

cov = covariation(x, y)
print("Ковариация:", cov)
corr = correlation_pearson(x, y)
print("Корреляция:", corr)


Ковариация: -2.5
Корреляция: -0.9999999999999998


## Регрессия

Метод наименьших квадратов

In [17]:
def regression_slope(n1: List[float], n2: List[float]) -> float:
    '''
    Коэффициент slope линейной регрессии

    Коэффициент корреляции по-сути определяет направление линии

    Args:
        n1 (List[float]): Первая последовательность
        n2 (List[float]): Вторая последовательность

    Returns:
        slope (float): Значение коэффициента
    '''

    slope = std(n2) / std(n1) * correlation_pearson(n1, n2)

    return slope


def regression_intercept(n1: List[float], n2: List[float]) -> float:
    '''
    Коэффициент intercept линейной регрессии

    Args:
        n1 (List[float]): Первая последовательность
        n2 (List[float]): Вторая последовательность

    Returns:
        intercept (float): Значение коэффициента
    '''

    intercept = mean(n2) - regression_slope * mean(n1)

    return intercept


def regression_linear(n1: List[float], n2: List[float], predictor: float) -> float:
    '''
    Предсказываемое значение линейной регрессии
    Уравнение линейной регрессии (x - предиктор, y - зависимая переменная):
    y = b0 + b1 * x

    Args:
        n1 (List[float]): Первая последовательность
        n2 (List[float]): Вторая последовательность
        predictor (float): Значение предиктора

    Returns:
        predicted_value (float): Предсказанное значение зависимой переменной

    '''
    
    predicted_value = regression_intercept(n1, n2) + regression_slope(n1, n2) * predictor

    return predicted_value

**Задача**

Есть зависимость двух количественных переменных X и Y. Рассчитайте коэффициент b1 для регрессионной прямой, если коэффициент детерминации равен 0,25:

MX = 15 (выборочное среднее), DX = 25

MY = 10, DY = 36

In [18]:
mean_x = 15
mean_y = 10
dx = 25
dy = 36
determination_coeff = 0.25

cr_pearson_coeff = determination_coeff ** 0.5
print("Correlation:", cr_pearson_coeff)

stdx = dx ** 0.5
stdy = dy ** 0.5

slope = (stdy / stdx) * cr_pearson_coeff
print("Slope:", slope)

intercept = mean_y - slope * mean_x
print("Intercept:", intercept)

Correlation: 0.5
Slope: 0.6
Intercept: 1.0
